# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest classifier, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [11]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [12]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract these items (one function for each): location, company, job title, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

In [13]:
### Rithika suggested I merge all of the find_alls into one large function, so the code below was created with her assistance. 

def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [14]:
parse(URL)

,Title,Location,Company,Salary,Synopsis
0,Data Scientist - Big Data & Analytics,"New York, NY 10154",KPMG,None,KPMG is currently seeking a Data Scientist - B...
1,Healthcare & Life Sciences - Data Scientist,"New York, NY 10154",KPMG,None,KPMG is currently seeking a Healthcare & Life ...
2,Entry Level – Research Analyst/Editor/Content ...,"New York, NY 10017 (Midtown area)","XG Consultants Group, Inc.",$15 an hour,Job Overview: XG Consultants Group is looking ...
3,Quantitative Analyst,"New York, NY",UBS,None,"Proficient using C++, VBA, R, SAS, SQL, Oracle..."
4,Data Science Intern - Fall 2017,"New York, NY 10016 (Gramercy area)",FactSet,None,"Scientist and engineer. Extract, transform, an..."
5,Data Science Intern - Fall 2017,"New York, NY 10016 (Gramercy area)",FactSet Research Systems,None,Just the right kind of work for a passionate d...
6,"Specialist, Advanced Insights - McKinsey Solut...","New York, NY 10022 (Midtown area)",McKinsey & Company,None,As one of the fastest-growing parts of our fir...
7,Machine Learning Engineer,"New York, NY",Indellient,None,"Big data and analytics, digital content delive..."
8,Machine Learning Engineer - KYC,"New York, NY",Bloomberg,None,Experience with Big data technologies such as ...
9,Quantitative Research Analyst,"New York, NY",Morningstar,None,Perform ad-hoc data cleaning and statistical a...


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).
##### Complete the following code to collect results from multiple cities and starting points.
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [15]:
YOUR_CITY = 'Washington%2C+DC'

In [16]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 2000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY, 
    'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose'
    'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
    'Memphis', 'Boston', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
    'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
    'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
    'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 'Pittsburgh', 
    'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 'Herndon',
    'Reston', 'Roanoke']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                synopsis = None
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 1000 == 0:  # Ram helped me build this counter to see how many. You can visibly see Ram's vernacular in the print statements.
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")    

You have 1000 results. 0 of these aren't rubbish.
You have 2000 results. 0 of these aren't rubbish.
You have 3000 results. 2 of these aren't rubbish.
You have 4000 results. 2 of these aren't rubbish.
You have 5000 results. 2 of these aren't rubbish.
You have 6000 results. 2 of these aren't rubbish.
You have 7000 results. 2 of these aren't rubbish.
You have 8000 results. 4 of these aren't rubbish.
You have 9000 results. 41 of these aren't rubbish.
You have 10000 results. 41 of these aren't rubbish.
You have 11000 results. 45 of these aren't rubbish.
You have 12000 results. 52 of these aren't rubbish.
You have 13000 results. 58 of these aren't rubbish.
You have 14000 results. 61 of these aren't rubbish.
You have 15000 results. 69 of these aren't rubbish.
You have 16000 results. 69 of these aren't rubbish.
You have 17000 results. 73 of these aren't rubbish.
You have 18000 results. 94 of these aren't rubbish.
You have 19000 results. 94 of these aren't rubbish.
You have 20000 results. 94 of

You have 156000 results. 475 of these aren't rubbish.
You have 157000 results. 476 of these aren't rubbish.
You have 158000 results. 476 of these aren't rubbish.
You have 159000 results. 476 of these aren't rubbish.
You have 160000 results. 476 of these aren't rubbish.
You have 161000 results. 476 of these aren't rubbish.
You have 162000 results. 476 of these aren't rubbish.
You have 163000 results. 476 of these aren't rubbish.
You have 164000 results. 477 of these aren't rubbish.
You have 165000 results. 488 of these aren't rubbish.
You have 166000 results. 497 of these aren't rubbish.
You have 167000 results. 501 of these aren't rubbish.
You have 168000 results. 501 of these aren't rubbish.
You have 169000 results. 501 of these aren't rubbish.
You have 170000 results. 504 of these aren't rubbish.
You have 171000 results. 506 of these aren't rubbish.
You have 172000 results. 506 of these aren't rubbish.
You have 173000 results. 506 of these aren't rubbish.
You have 174000 results. 506

In [17]:
df_more.to_csv('Indeed_Project_3_df_more_long_not_cleaned.csv', encoding='utf-8')

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [18]:
print (df_more.head())
print (df_more.shape)
print (df_more[df_more.Salary != 'None'].shape)
df_more = df_more[df_more.Salary != 'None'].drop_duplicates().dropna()
print (df_more.shape)

                             Title           Location                 Company  \
0            Systems Administrator         Forest, VA  Innerspec Technologies   
1          Environmental Scientist     Blacksburg, VA             CyberCoders   
2                   Senior Manager      United States                Exponent   
3  Software Engineering Specialist  Roanoke, VA 24019        General Electric   
4       Sr Staff Software Engineer  Roanoke, VA 24019        General Electric   

  Salary                                           Synopsis  
0   None  Manage backup and restore services to ensure t...  
1   None  Environmental Scientist If you are an Environm...  
2   None  Providing case management, data processing, an...  
3   None  You will work with a group of energized and fo...  
4   None  Architects, Data Scientists, Businesses & Prod...  
(199055, 5)
(199055, 5)
(581, 5)


In [19]:
df_more = df_more[df_more.Salary.str.contains("hour") == False]
df_more = df_more[df_more.Salary.str.contains("month") == False]
print (df_more.shape)
df_more.head()

(409, 5)


,Title,Location,Company,Salary,Synopsis
2724,Environmental Consultant,"Nashville, TN 37220",LP Environmental,"$50,000 - $100,000 a year",We are seeking a mid-level Environmental Scien...
8054,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,"$125,000 a year",We are currently looking for an experienced Da...
8194,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,"$55,000 - $60,000 a year",Produce analyses of crime trends and provide d...
8197,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,"$55,000 - $60,000 a year",(3) analyzing qualitative and quantitative dat...
8234,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,"$85,000 - $125,000 a year",Our startup client working on risk and fraud d...


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [23]:
def salary_stripper(dataframe, column):
    dataframe[str(column)] = dataframe[str(column)].replace({'\$':''}, regex = True)
    dataframe[str(column)].replace(regex=True,inplace=True,to_replace=r'\D',value=r' ')
    dataframe[str(column)] = dataframe[str(column)].str.replace(' ',',')
    dataframe = dataframe.join(dataframe[str(column)].str.split(',,,', 1, expand=True).rename(columns={0:'Low', 1:'High'}))
    dataframe['Low'] = dataframe['Low'].str.replace(',','')
    dataframe['Low'] = dataframe['Low'].astype('float64')
    dataframe.drop(str(column), axis=1, inplace=True)
    dataframe['High'] = dataframe['High'].str.replace(',','')
    dataframe['High'] = dataframe['High'].apply(pd.to_numeric)
    dataframe['Average'] = dataframe[['Low', 'High']].mean(axis=1)
    return dataframe

In [25]:
df_more = salary_stripper(df_more, 'Salary')

In [29]:
df_more[50:60]

,Title,Location,Company,Synopsis,Low,High,Average
17086,Senior Data Scientist Healthcare,"Philadelphia, PA",Harnham,Senior Data Scientist - Healthcare. Reporting ...,170000.0,NaN,170000.0
17252,Data Scientist,"Philadelphia, PA",Smith Hanley Associates,Data Scientist Qualifications:. Data Scientist...,120000.0,140000.0,130000.0
17287,Predictive Analytics (Machine Learning),"Wilmington, DE","Kennedy Unlimited Inc, Professional Staffing",Salary 130K to 140K We are assisting our Clien...,130000.0,140000.0,135000.0
17376,Data Scientist/Machine Learning Specialist,"Philadelphia, PA",Jobspring Partners,"A leading healthcare analytical team, one of t...",100000.0,160000.0,130000.0
17406,NMR Spectroscopist,"West Point, PA",EPM Scientific,Scientist | Structure Elucidation*. Excellent ...,115000.0,145000.0,130000.0
17421,Data Scientist/Optimization Engineer,"Philadelphia, PA",Jobspring Partners,In this role you will be extracting power plan...,100000.0,125000.0,112500.0
17451,"Research Scientist (Gene Therapy, Molecular Cl...","Philadelphia, PA",Apex Life Sciences,Analyze and interpret the results including st...,50000.0,NaN,50000.0
17459,Machine Learning Researcher,"Philadelphia, PA",Jobspring Partners,"Experience with Data Mining. This agency, spec...",100000.0,130000.0,115000.0
17467,Quant Research Analyst,"Philadelphia, PA",3coast,Experience in data and time series analysis st...,150000.0,250000.0,200000.0
17481,Quantitative Analyst,"Philadelphia, PA",Winston Fox,"In this role, you will examine global markets,...",100000.0,110000.0,105000.0


In [30]:
df_more = df_more.join(df_more['Location'].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))


In [31]:
df_more.head()

,Title,Location,Company,Synopsis,Low,High,Average,City,State
2724,Environmental Consultant,"Nashville, TN 37220",LP Environmental,We are seeking a mid-level Environmental Scien...,50000.0,100000.0,75000.0,Nashville,TN 37220
8054,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,We are currently looking for an experienced Da...,125000.0,NaN,125000.0,Chicago,IL 60601 (Loop area)
8194,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,Produce analyses of crime trends and provide d...,55000.0,60000.0,57500.0,Chicago,IL
8197,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,(3) analyzing qualitative and quantitative dat...,55000.0,60000.0,57500.0,Chicago,IL 60606 (Loop area)
8234,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,Our startup client working on risk and fraud d...,85000.0,125000.0,105000.0,Chicago,IL


### Save your results as a CSV

In [32]:
df_more.to_csv('Indeed_Project_3_df_more_long.csv', encoding='utf-8')

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the job titles.
- Build a new random forest model with location and these new features included.

In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [ ]:
## YOUR CODE HERE

#### Repeat the model-building process with a non-tree-based method.

In [ ]:
## YOUR CODE HERE

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the job descriptions. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE